In [2]:
# read data
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
# first 400 characters
print(text[:400])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it 


In [5]:
# all the unique characters on our data
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# create a mapping
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # tokenize
decode = lambda l: ''.join([itos[i] for i in l]) # detokenize

print(encode("awsome"))
print(decode(encode("awsome")))

[39, 61, 57, 53, 51, 43]
awsome


In [7]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape,data.dtype)
print(data[:400])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [8]:
# Train and validation split
n = int(0.8*len(data)) #%80 test, %20 val
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [11]:
torch.manual_seed(41)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print(f"Inputs = {xb.shape}, \n{xb}")
print(f"Targets = {yb.shape}, \n{yb}")

print('==========')

for b in range(batch_size): # batch 
    for t in range(block_size): # time 
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

Inputs = torch.Size([4, 8]), 
tensor([[43,  1, 52, 53, 58, 11,  1, 44],
        [53, 52, 43, 57, 58, 63,  6,  0],
        [ 1, 49, 47, 50, 50, 57,  1, 51],
        [14, 43, 44, 53, 56, 43,  1, 57]])
Targets = torch.Size([4, 8]), 
tensor([[ 1, 52, 53, 58, 11,  1, 44, 53],
        [52, 43, 57, 58, 63,  6,  0, 32],
        [49, 47, 50, 50, 57,  1, 51, 63],
        [43, 44, 53, 56, 43,  1, 57, 53]])
when input is [43] the target: 1
when input is [43, 1] the target: 52
when input is [43, 1, 52] the target: 53
when input is [43, 1, 52, 53] the target: 58
when input is [43, 1, 52, 53, 58] the target: 11
when input is [43, 1, 52, 53, 58, 11] the target: 1
when input is [43, 1, 52, 53, 58, 11, 1] the target: 44
when input is [43, 1, 52, 53, 58, 11, 1, 44] the target: 53
when input is [53] the target: 52
when input is [53, 52] the target: 43
when input is [53, 52, 43] the target: 57
when input is [53, 52, 43, 57] the target: 58
when input is [53, 52, 43, 57, 58] the target: 63
when input is [53,

In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        
        for _ in range(max_new_tokens):
            
            logits, loss = self(idx)
            logits = logits[:, -1, :] 
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.4730, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [13]:
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-3)

In [14]:
batch_size = 32
for steps in range(10000):
    xb,yb = get_batch('train')

    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.496304988861084


In [15]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


Iyoteng h hasbe pan hatrance
Rie hicomyonthar's
PAS:
AKI tith henouratucenonthioneir thondy, y helti


**Math Trick in self-attention**

In [18]:
torch.manual_seed(41)

B,T,C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [19]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # t,c 
        xbow[b,t] = torch.mean(xprev,0)

In [20]:
x[0]

tensor([[ 0.2465, -0.4717],
        [-1.5184, -0.2124],
        [ 0.5660, -1.6637],
        [ 0.3338,  1.6051],
        [-0.6919, -0.5399],
        [-0.9585, -0.5706],
        [-0.5434, -0.4347],
        [ 0.4036, -0.3977]])

In [21]:
xbow[0]

tensor([[ 0.2465, -0.4717],
        [-0.6359, -0.3420],
        [-0.2353, -0.7826],
        [-0.0930, -0.1857],
        [-0.2128, -0.2565],
        [-0.3371, -0.3088],
        [-0.3665, -0.3268],
        [-0.2703, -0.3357]])

In [30]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1,keepdim=True)
xbow2= wei @ x # (B,T,T) @ (B,T,C) -> (B,T,C)
torch.allclose(xbow,xbow2)

True

In [32]:
xbow[1],xbow2[1]

(tensor([[-7.5931e-01, -9.4012e-01],
         [-2.3493e-01, -6.6656e-01],
         [-1.0679e-01, -9.2614e-01],
         [ 6.6689e-04, -6.2734e-01],
         [-3.7111e-01, -3.6672e-01],
         [-2.0627e-01, -4.3221e-01],
         [-1.3202e-01, -1.9720e-01],
         [-2.4754e-01,  6.6256e-02]]),
 tensor([[-7.5931e-01, -9.4012e-01],
         [-2.3493e-01, -6.6656e-01],
         [-1.0679e-01, -9.2614e-01],
         [ 6.6689e-04, -6.2734e-01],
         [-3.7111e-01, -3.6672e-01],
         [-2.0627e-01, -4.3221e-01],
         [-1.3202e-01, -1.9720e-01],
         [-2.4754e-01,  6.6256e-02]]))

In [ ]:
#54:42